In [1]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
import os, re, time, json, random
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver import DesiredCapabilities


In [2]:
# make chrome log requests
capabilities = DesiredCapabilities.CHROME
capabilities["goog:loggingPrefs"] = {"performance": "ALL"}  # chromedriver 75+
# starts browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), desired_capabilities=capabilities)
driver.implicitly_wait(10)




====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\USER\.wdm\drivers\chromedriver\win32\101.0.4951.41]


In [39]:
mobile = '9910737626'
login_url = 'https://app.qandi.com/login'
mobile_email_xp = '//*[@id="mobile_num"]'
next_btn_xp = '//*[@id="mobile-input-btn"]'
enter_otp_xp = '//*[@id="otp_num"]'
verify_xp = '//*[@id="otp-verify-btn"]'
custom_url = 'https://app.qandi.com/exam_custom'

sub_xp = '//*[@id="Mathematics_btn"]'
save_and_next_xp_f = '//*[@id="question_section"]/div[3]/a[1]'
save_and_next_xp_o = '//*[@id="question_section"]/div/div[3]/a[1]'
q_type_xp = '//*[@id="question_section"]/div/div[1]/span'
sub_test_xp = '//*[@id="bt-modal-confirm_over"]'
end_exam_xp = '//*[@id="endExam"]/div/div'

wait = 3
prob = 1 # for toppers


In [4]:
q_data = pd.read_excel('all_live_questions.xlsx')
# q_data[:1]

In [5]:
# Login to qandi

def login():
    driver.get(login_url)

    driver.find_element(By.ID, 'mobile_num').send_keys(mobile)
    time.sleep(1)
    next_btn_elm = driver.find_element(By.ID, 'mobile-input-btn')

    try:
        next_btn_elm.click()
        enter_otp_elm = driver.find_element(By.ID, 'otp_num')
        otp = input('Please input otp and then press enter to continue:')
        enter_otp_elm.send_keys(otp)
        try:
            verify_elm = driver.find_element(By.ID, 'otp-verify-btn')
            driver.execute_script("arguments[0].click();", verify_elm) 
        except:
            verify_elm.click()
                   
    except:
        try:
            driver.execute_script("arguments[0].click();", next_btn_elm)
            enter_otp_elm = driver.find_element(By.ID, 'otp_num')
        except:
            print('Not able to login')
    return
    

In [16]:
login()

Please input otp and then press enter to continue: 83126


In [7]:

def get_topic_ids(chp_id):
    chap_box_content = driver.find_element(By.ID, f'chapter_box_{chp_id}').get_attribute('innerHTML')
    topic_ids = re.findall(r'id="chpt_topic_(\d+)"', chap_box_content)
    topic_ids = [int(i) for i in topic_ids]
    return topic_ids


In [17]:

def get_curr_q_id():
    curr_q_xp = '//*[@id="current_question"]'
    curr_q_elm = driver.find_element(By.XPATH, curr_q_xp)
    curr_q_id = curr_q_elm.get_attribute('value')    
    return int(curr_q_id)
# get_curr_q_id()


In [34]:

def get_corr_options_or_ans(q_id, prob):
    isItNumQuestion = int(q_data[q_data['question_id']==q_id]['template_type'].values[0])==11
    if isItNumQuestion:
        ans = re.sub('[\[\]\"]', '', q_data[q_data['question_id']==curr_q_id]['answers'].values[0])
        if random.random() > prob:
            ans += 1
        return ans
    else:
        ans_text = q_data[q_data['question_id']==q_id]['answers'].values[0]
        ans_text_dict = json.loads(ans_text)
        corr_options = list(ans_text_dict.keys())
        corr_options = [int(i) for i in corr_options]
        incorr_options = [i for i in range(1,5) if i not in corr_options]
        if random.random() > prob:
            corr_options[0] = incorr_options[0]
        return corr_options



In [112]:

def do_question(o_xps, save_and_next_xp, wait, curr_q_id):
    # choose option
    
    for o_xp in o_xps:
        o_xp_elm = driver.find_element(By.XPATH, o_xp)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", o_xp_elm)
    # click save and next after wait time is over
    time.sleep(wait)
    save_and_next_elm = driver.find_element(By.XPATH, save_and_next_xp)
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", save_and_next_elm)
    
    time.sleep(1)
    curr_id_temp = get_curr_q_id()
    qoption_err_xp = f'//*[@id="qoption_err_{curr_id_temp}"]'
    qoption_err_elm = driver.find_element(By.XPATH, qoption_err_xp)

    if curr_id_temp == curr_q_id :
        if qoption_err_elm.value_of_css_property('display') == 'none':
            time.sleep(1)
            driver.execute_script("arguments[0].click();", save_and_next_elm)
        # save_and_next_elm.click()
        else:
            for o_xp in o_xps:
                o_xp_elm = driver.find_element(By.XPATH, o_xp)
                time.sleep(1)
                driver.execute_script("arguments[0].click();", o_xp_elm)
            # click save and next after wait time is over
            time.sleep(wait)
            save_and_next_elm = driver.find_element(By.XPATH, save_and_next_xp)
            time.sleep(0.5)
            driver.execute_script("arguments[0].click();", save_and_next_elm)
    return


In [20]:

def do_question_num(ans, save_and_next_xp, wait, curr_q_id):
    
    
    num_ans_box_xp = f'//*[@id="quest_option_{curr_q_id}"]'
    num_ans_box_elm = driver.find_element(By.XPATH, num_ans_box_xp)
    num_ans_box_elm.send_keys(ans)
    time.sleep(wait)
    save_and_next_elm = driver.find_element(By.XPATH, save_and_next_xp)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", save_and_next_elm)
    return 


In [110]:
# Attempt Topic test

def do_topic_test():
    
    # Do first question
    curr_q_id = get_curr_q_id()
    # print(curr_q_id)
    q_type_elm = driver.find_element(By.XPATH, q_type_xp)
    q_type = q_type_elm.get_attribute('innerText')
    try:
        if q_type == 'NUMERICAL' or int(q_data[q_data['question_id']==curr_q_id]['template_type'].values[0])==11:
            ans = get_corr_options_or_ans(curr_q_id, prob)
            do_question_num(ans, save_and_next_xp_f, wait, curr_q_id)
        else:
            corr_options = get_corr_options_or_ans(curr_q_id, prob)
            count = 0
            o_xps = []
            for corr_option in corr_options:
                o_xp_f = f'//*[@id="question_section"]/div[2]/div[2]/div[{corr_option}]/div/label'
                o_xps.append(o_xp_f)

            do_question(o_xps, save_and_next_xp_f, wait, curr_q_id)
            
    except StaleElementReferenceException:
        if q_type == 'NUMERICAL':
            input(f'Do the question with id: {curr_q_id} and ans as: {ans}')
        else:
            input(f'Do the question with id: {curr_q_id} and correct option as as: {corr_options}')
        
    count += 1

    # Do other questions
    sub_test_elm = driver.find_element(By.XPATH, sub_test_xp)
    test_end_condition = sub_test_elm.get_attribute('innerText')

    while test_end_condition != 'SUBMIT TEST':
        
        curr_q_id = get_curr_q_id()
        q_type_elm = driver.find_element(By.XPATH, q_type_xp)
        q_type = q_type_elm.get_attribute('innerText')
        try:
            if q_type == 'NUMERICAL' or int(q_data[q_data['question_id']==curr_q_id]['template_type'].values[0])==11:
                ans = get_corr_options_or_ans(curr_q_id, prob)
                do_question_num(ans, save_and_next_xp_o, wait, curr_q_id)
            else:
                o_xps = []
                # try:
                corr_options = get_corr_options_or_ans(curr_q_id, prob)
                for corr_option in corr_options:
                    o_xp_o = f'//*[@id="question_section"]/div/div[2]/div[2]/div[{corr_option}]/div/label'
                    o_xps.append(o_xp_o)
                time.sleep(1)
                do_question(o_xps, save_and_next_xp_o, wait, curr_q_id)
        except StaleElementReferenceException:
            if q_type == 'NUMERICAL':
                input(f'Do the question with id: {curr_q_id} and ans as: {ans}')
            else:
                input(f'Do the question with id: {curr_q_id} and correct option as as: {corr_options}')
        count += 1
        test_end_condition = sub_test_elm.get_attribute('innerText')
    time.sleep(1)
    if False:
        
        end_exam_elm = driver.find_element(By.XPATH, end_exam_xp)
        SUB_TEST_elm = driver.find_element(By.ID, 'bt-modal-confirm_over')
        time.sleep(1)
        driver.execute_script("arguments[0].click();", SUB_TEST_elm)
    else:
        input('Please submit the exam.')
    # driver.save_screenshot(os.path.join(os.getcwd(),'botTestOutput',f'{Subject[0]}_{Topic}.png'))
    return 


In [22]:

def start_topic_test(topic_id):
    
    topic_select_elm = driver.find_element(By.ID, f"chpt_topic_{topic_id}")
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", topic_select_elm)
    take_test_for_sel_topic_elm = driver.find_element(By.XPATH, '//*[@id="topic_custom_footer"]/button')
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", take_test_for_sel_topic_elm)
    time.sleep(1)
    go_for_it_elm = driver.find_element(By.XPATH, '//*[@id="goto-exam-btn"]')
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", go_for_it_elm)

    test_instruction_elm = driver.find_element(By.ID, 'test_instruction')
    driver.execute_script("arguments[0].click();", go_for_it_elm)
    time.sleep(0.5)
    while test_instruction_elm.is_displayed():
        driver.execute_script("arguments[0].click();", go_for_it_elm)
        test_instruction_elm = driver.find_element(By.ID, 'test_instruction')
        print('At instruction page')
        time.sleep(0.5)
    return


In [79]:

def expand_chapter(chp_id):
    driver.get(custom_url)
    time.sleep(1)
    chp_expand_elm = driver.find_element(By.ID, f"expand_topic_{chp_id}")
    time.sleep(0.5)
    driver.execute_script("arguments[0].click();", chp_expand_elm)
    time.sleep(0.5)
    return 


In [24]:
# For math chapter ids

driver.get(custom_url)

math_list_innerHTML = driver.find_element(By.ID,'Mathematics_list').get_attribute('innerHTML')
math_chp_ids = re.findall(r'id="chapter_box_(\d+)"', math_list_innerHTML)
math_chp_ids = [int(i) for i in math_chp_ids]


In [ ]:
# For chemistry chapter ids

driver.get(custom_url)

chem_list_innerHTML = driver.find_element(By.ID,'Chemistry_list').get_attribute('innerHTML')
chem_chp_ids = re.findall(r'id="chapter_box_(\d+)"', chem_list_innerHTML)
chem_chp_ids = [int(i) for i in chem_chp_ids]


In [ ]:
# For Physics

driver.get(custom_url)

phy_list_innerHTML = driver.find_element(By.ID,'Physics_list').get_attribute('innerHTML')
phy_chp_ids = re.findall(r'id="chapter_box_(\d+)"', phy_list_innerHTML)
phy_chp_ids = [int(i) for i in phy_chp_ids]


In [115]:

# Adaptive test starts

chp_no = 0
for chp_id in math_chp_ids[12:13]:
    time.sleep(0.5)
    expand_chapter(chp_id)
    topic_ids = get_topic_ids(chp_id)
    topic_no = 0
    for topic_id in topic_ids:
        
        start_topic_test(topic_id)
        do_topic_test()        
        expand_chapter(chp_id)
        topic_no+=1
        print(f'Topics completed: {topic_no}')
    
    chp_no+=1
    print(f'Chapters completed: {chp_no}')
    

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=101.0.4951.64)
Stacktrace:
Backtrace:
	Ordinal0 [0x0039B8F3+2406643]
	Ordinal0 [0x0032AF31+1945393]
	Ordinal0 [0x0021C748+837448]
	Ordinal0 [0x0021144A+791626]
	Ordinal0 [0x002124B2+795826]
	Ordinal0 [0x0020BF09+769801]
	Ordinal0 [0x0021DAC0+842432]
	Ordinal0 [0x00273E62+1195618]
	Ordinal0 [0x00264096+1130646]
	Ordinal0 [0x0023E636+976438]
	Ordinal0 [0x0023F546+980294]
	GetHandleVerifier [0x00609612+2498066]
	GetHandleVerifier [0x005FC920+2445600]
	GetHandleVerifier [0x00434F2A+579370]
	GetHandleVerifier [0x00433D36+574774]
	Ordinal0 [0x00331C0B+1973259]
	Ordinal0 [0x00336688+1992328]
	Ordinal0 [0x00336775+1992565]
	Ordinal0 [0x0033F8D1+2029777]
	BaseThreadInitThunk [0x77703744+36]
	RtlSetCurrentTransaction [0x77DC9E54+212]
	RtlSetCurrentTransaction [0x77DC9E1F+159]


In [ ]:
chp_id = math_chp_ids[12]
topic_ids = get_topic_ids(chp_id)
topic_ids

In [93]:
o1_xp = '//*[@id="question_section"]/div/div[2]/div[2]/div[1]'
o1_elm = driver.find_element(By.XPATH, o1_xp)


In [105]:
qoption_err_xp = '//*[@id="qoption_err_177874"]'
qoption_err_elm = driver.find_element(By.XPATH, qoption_err_xp)
qoption_err_elm.value_of_css_property('display')
# get_attribute('bs-blue')
# get_property('display')


'none'

In [ ]:
type(logs_text)

In [51]:
len(logs)

69508

In [ ]:
logs = driver.get_log("performance")
logs